In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("../Datasets/Plane Crashes.csv")

In [3]:
df["Survivors"] = df["Crew on board"] + df["Pax on board"] - df["Crew fatalities"] - df["PAX fatalities"]

In [4]:
flight_types_mapping = {
    "Test": "testing/training",
    "Training": "testing/training",
    "Calibration": "testing/training",
    "Delivery": "delivery",
    "Postal (mail)": "delivery",
    "Supply": "delivery",
    "Cargo": "delivery",
    "Fire fighting": "support",
    "Refuelling": "support",
    "Spraying (Agricultural)": "support",
    "Ambulance": "support",
    "Ferry": "support",
    "Topographic": "support",
    "Humanitarian": "support",
    "Calibration": "support",
    "Meteorological / Weather": "support",
    "Geographical / Geophysical / Scientific": "support",
    "Aerial photography": "private/business",
    "Scheduled Revenue Flight": "private/business",
    "Private": "private/business",
    "Charter/Taxi (Non Scheduled Revenue Flight)": "private/business",
    "Demonstration": "private/business",
    "Executive/Corporate/Business": "private/business",
    "Cinematography": "private/business",
    "Skydiving / Paratroopers": "private/business",
    "Aerobatic": "private/business",
    "Positioning": "private/business",
    "Military": "government",
    "Government": "government",
    "Survey / Patrol / Reconnaissance": "government",
    "Illegal (smuggling)": "illegal",
    "Bombing": "illegal"
}

In [5]:
df.dropna(subset=["Flight type"], inplace=True)
df["flight_group"] = df["Flight type"].map(flight_types_mapping)

In [22]:
df_crash_cause = pd.DataFrame()
for group in df["flight_group"].unique():
    df_tmp = pd.DataFrame()
    df_tmp["group"] = [group]
    for crash_cause in df["Crash cause"].unique():
        df_tmp[crash_cause] = [len(df[(df["Crash cause"] == crash_cause) & (df["flight_group"] == group)])]
    df_crash_cause = pd.concat([df_crash_cause,df_tmp])

In [23]:
df_crash_cause

,group,Technical failure,Unknown,Weather,Human factor,Other causes,"Terrorism act, Hijacking, Sabotage"
0,testing/training,836,1457,105,1399,51,17
0,government,1188,2748,337,961,82,475
0,delivery,1011,949,259,1366,152,178
0,support,353,438,78,713,43,24
0,private/business,2712,3141,739,5285,293,192
0,illegal,101,213,10,191,6,376
